In [1]:
# Set the environment path to find NLP
import sys
sys.path.append("../../")
import time
import logging
import csv
import os
import pandas as pd
import shutil
import numpy as np
import torch
import sys
from scipy.stats import pearsonr
from scipy.spatial import distance
from sklearn.externals import joblib

# Import utils
from utils_nlp.azureml import azureml_utils
from utils_nlp.dataset import stsbenchmark
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)

# Tensorflow dependencies for Google Universal Sentence Encoder
import tensorflow as tf
import tensorflow_hub as hub
tf.logging.set_verbosity(tf.logging.ERROR) # reduce logging output

# AzureML packages
import azureml as aml
import logging
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)
from azureml.train.automl import AutoMLConfig
from azureml.core import Datastore, Experiment
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.automl import AutoMLStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

print("System version: {}".format(sys.version))
print("Azure ML SDK Version:", aml.core.VERSION)
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow Version:", tf.VERSION)

BASE_DATA_PATH = '../../data'

ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>"
)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

experiment_name = 'automl-sentence-similarity'
project_folder = './automl-sentence-similarity'

experiment = Experiment(ws, experiment_name)
experiment

from azureml.core.compute import ComputeTarget, AmlCompute
# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(compute_target.get_status().serialize())

# Load in the raw datasets as pandas dataframes
train_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")
dev_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="dev")
test_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="test")

# Clean each dataset by lowercasing text, removing irrelevant columns,
# and renaming the remaining columns
train = stsbenchmark.clean_sts(train_raw)
dev = stsbenchmark.clean_sts(dev_raw)
test = stsbenchmark.clean_sts(test_raw)

if not os.path.isdir('data'):
    os.mkdir('data')
    
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

train.to_csv("data/train.csv", index=False)
# test.to_csv("data/test.csv", index=False)
dev.to_csv("data/dev.csv", index=False)


W0618 16:45:51.878871 26704 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Turning diagnostics collection on. 
System version: 3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)]
Azure ML SDK Version: 1.0.41
Pandas version: 0.23.4
Tensorflow Version: 1.13.1


W0618 16:46:02.219784 26704 authentication.py:494] Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Performing interactive authentication. Please follow the instructions on the terminal.


W0618 16:46:02.576774 28512 _profile.py:1082] Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
W0618 16:46:09.812178 26704 _profile.py:774] You have logged in. Now let us find all the subscriptions to which you have access...


...

Data downloaded to ../../data\raw\stsbenchmark


100%|████████████████████████████████████████████████| 401/401 [00:01<00:00, 255KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|████████████████████████████████████████████████| 401/401 [00:01<00:00, 246KB/s]


Data downloaded to ../../data\raw\stsbenchmark


In [2]:
from azureml.data.data_reference import DataReference      

ds = ws.datastores['workspacefilestore']#.get_default_datastore()
ds.upload(src_dir='./data', target_path='stsbenchmark_data', overwrite=True, show_progress=True)

input_data = DataReference(datastore=ds, 
                           data_reference_name="stsbenchmark",
                           path_on_datastore='stsbenchmark_data/',
                           overwrite=False)

Uploading ./data\dev.csv
Uploading ./data\train.csv
Uploaded ./data\dev.csv, 1 files out of an estimated total of 2
Uploaded ./data\train.csv, 2 files out of an estimated total of 2


aml_run_config = RunConfiguration()

# Use the aml_compute you created above. 
aml_run_config.target = aml_compute

# Enable Docker
aml_run_config.environment.docker.enabled = True

# Set Docker base image to the default CPU-based image
aml_run_config.environment.docker.base_image = "mcr.microsoft.com/azureml/base:0.2.1"

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
aml_run_config.environment.python.user_managed_dependencies = False

# Auto-prepare the Docker image when used for execution (if it is not already prepared)
aml_run_config.auto_prepare_environment = True

# Specify CondaDependencies obj, add necessary packages
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn'], 
    pip_packages=['azureml-sdk', 'azureml-dataprep', 'azureml-train-automl==1.0.33'], 
    pin_sdk_version=False)

print ("Run configuration created.")

In [3]:
# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target

conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = aml.core.runconfig.DEFAULT_CPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
conda_run_config.environment.python.user_managed_dependencies = False

conda_run_config.environment.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'azureml-sdk', 'azureml-dataprep', 'azureml-train-automl==1.0.33'], 
                              conda_packages=['numpy', 'py-xgboost', 'pandas', 'tensorflow', 'tensorflow-hub', 'scikit-learn'], 
                              pin_sdk_version=False)


print('run config is ready')

run config is ready


In [4]:
embedded_data = PipelineData("embedded_data", datastore=ds)

In [5]:
%%writefile $project_folder/embed.py
import argparse
import os
import azureml.core
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
tf.logging.set_verbosity(tf.logging.ERROR) # reduce logging output

def google_encoder(dataset):
    """ Function that embeds sentences using the Google Universal
    Sentence Encoder pretrained model
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    emb1: 512-dimensional representation of sentence1
    emb2: 512-dimensional representation of sentence2
    """
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    # Apply embedding model and normalize the input
    sts_encode1 = tf.nn.l2_normalize(embedding_model(sts_input1), axis=1)
    sts_encode2 = tf.nn.l2_normalize(embedding_model(sts_input2), axis=1)
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        emb1, emb2 = session.run(
          [sts_encode1, sts_encode2],
          feed_dict={
              sts_input1: dataset['sentence1'],
              sts_input2: dataset['sentence2']
          })
    return emb1, emb2

def feature_engineering(dataset):
    """Extracts embedding features from the dataset and returns
    features and target in a dataframe
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    df: pandas dataframe with embedding features
    scores: list of target variables
    """
    google_USE_emb1, google_USE_emb2 = google_encoder(dataset)
    n_google = google_USE_emb1.shape[1] #length of the embeddings 
    df = np.concatenate((google_USE_emb1, google_USE_emb2), axis=1)
    names = ['USEEmb1_'+str(i) for i in range(n_google)]+['USEEmb2_'+str(i) for i in range(n_google)]
    df = pd.DataFrame(df, columns=names)
    return df, dataset['score']

def write_output(df, path, name):
    os.makedirs(path, exist_ok=True)
    print("%s created" % path)
    df.to_csv(path + "/" + name, index=False)

parser = argparse.ArgumentParser()
parser.add_argument("--sentence_data", type=str)
parser.add_argument("--embedded_data", type=str)
args = parser.parse_args()

# Import the Universal Sentence Encoder's TF Hub module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embedding_model = hub.Module(module_url)

train = pd.read_csv(args.sentence_data + "/train.csv")
dev = pd.read_csv(args.sentence_data + "/dev.csv")

training_data, training_scores = feature_engineering(train)
validation_data, validation_scores = feature_engineering(dev)

write_output(training_data, args.embedded_data, "X_train.csv")
write_output(pd.DataFrame(training_scores, columns=['score']), args.embedded_data, "y_train.csv")

write_output(validation_data, args.embedded_data, "X_dev.csv")
write_output(pd.DataFrame(validation_scores, columns=['score']), args.embedded_data, "y_dev.csv")

Overwriting ./automl-sentence-similarity/embed.py


In [6]:
embedStep = PythonScriptStep(
    name="Embed",
    script_name="embed.py", 
    arguments=["--embedded_data", embedded_data,
               "--sentence_data", input_data],
    inputs=[input_data],
    outputs=[embedded_data],
    compute_target=compute_target,
    runconfig = conda_run_config,
    hash_paths=["embed.py"],
    source_directory=project_folder,
    allow_reuse=True
)

In [7]:
%%writefile $project_folder/get_data.py

import os
import pandas as pd

def get_data():
    print("In get_data")
    print(os.environ['AZUREML_DATAREFERENCE_embedded_data'])
    X_train  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/X_train.csv")
    y_train  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/y_train.csv")
    X_dev  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/X_dev.csv")
    y_dev  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/y_dev.csv")
    
    return { "X" : X_train.values, "y" : y_train.values.flatten(), "X_valid": X_dev.values, "y_valid": y_dev.values.flatten()}

Overwriting ./automl-sentence-similarity/get_data.py


In [8]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 5,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
}
automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             path = project_folder,
                             compute_target=compute_target,
                             run_configuration=conda_run_config,
                             data_script = project_folder + "/get_data.py",
                             **automl_settings
                            )

In [9]:
from azureml.pipeline.core import PipelineData, TrainingOutput
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [10]:
from azureml.train.automl import AutoMLStep

automl_step = AutoMLStep(
    name='automl_step',
    automl_config=automl_config,
    inputs=[embedded_data],
    outputs=[metrics_data, model_data],
    hash_paths=["get_data.py"],
    allow_reuse=True)

In [11]:
from azureml.pipeline.core import Pipeline

automl_step.run_after(embedStep)
pipeline = Pipeline(
    description="pipeline_embed_automl",
    workspace=ws,    
    steps=[automl_step])

In get_data


In [12]:
pipeline_run = experiment.submit(pipeline)

Created step automl_step [db9288ed][21fb4afa-ca46-4929-b39e-49a272ff0cbd], (This step is eligible to reuse a previous run's output)
Created step Embed [70352e68][d271deed-bd3b-4e41-9814-29fc11e585b4], (This step is eligible to reuse a previous run's output)
Using data reference stsbenchmark for StepId [a1aa29ca][e3340790-c54f-4147-8dd0-bcb80a9b7b46], (Consumers of this data are eligible to reuse prior runs.)
Submitted pipeline run: 50a80cb2-8adb-4cd5-a337-c493404b7549


In [13]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', '…

In [ ]:
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 50a80cb2-8adb-4cd5-a337-c493404b7549
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/automl-sentence-similarity/runs/50a80cb2-8adb-4cd5-a337-c493404b7549
PipelineRun Status: Running


StepRunId: f78cb325-802a-4779-ada8-05db82c97835
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/automl-sentence-similarity/runs/f78cb325-802a-4779-ada8-05db82c97835
StepRun( Embed ) Status: NotStarted

Streaming azureml-logs/20_image_build_log.txt
StepRun( Embed ) Status: Running
2019/06/18 20:46:34 Downloading source code...
2019/06/18 20:46:35 Finished downloading source code
2019/06/18 20:46:35 Using acb_vol_35e6dda2-50a4-4121-9aa7-690a7b1c1e12 as the home volume
2019/06/18 20:46:35 Cr

pip-19.1.1           | 1.8 MB    |            |   0% 
pip-19.1.1           | 1.8 MB    | #######8   |  79% 
pip-19.1.1           | 1.8 MB    | #########4 |  94% 
pip-19.1.1           | 1.8 MB    | ########## | 100% 

tensorboard-1.13.1   | 3.3 MB    |            |   0% 
tensorboard-1.13.1   | 3.3 MB    | #######6   |  76% 
tensorboard-1.13.1   | 3.3 MB    | #########9 |  99% 
tensorboard-1.13.1   | 3.3 MB    | ########## | 100% 

werkzeug-0.15.4      | 254 KB    |            |   0% 
werkzeug-0.15.4      | 254 KB    | #########7 |  97% 
werkzeug-0.15.4      | 254 KB    | ########## | 100% 

python-3.6.2         | 19.0 MB   |            |   0% 
python-3.6.2         | 19.0 MB   | ##7        |  27% 
python-3.6.2         | 19.0 MB   | ######3    |  64% 
python-3.6.2         | 19.0 MB   | ########   |  80% 
python-3.6.2         | 19.0 MB   | #########2 |  92% 
python-3.6.2         | 19.0 MB   | ########## | 100% 

gast-0.2.2           | 10 KB     |            |   0% 
gast-0.2.2           | 1

  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/ee/30/5c/3a3b5e1386c8db9a3be5f5c3933644ae0533c1351c6a8eb4b5
  Stored in directory: /root/.cache/pip/wheels/e8/74/51/42c2d41c02bdc6f0e604476b7e4293b8c98d0bcbfa1dff78c8
  Stored in directory: /root/.cache/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
  Stored in directory: /root/.cache/pip/wheels/6b/b9/15/c8c6a1e095a370e8c3273e65a5c982e5cf355dde16d77502f5
Successfully built dill pathspec JsonSir JsonForm smart-open PyYAML pycparser pyrsistent
ERROR: azureml-train-core 1.0.43 has requirement azurem

Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_361db214fdeff5eb353cb3d3027c9e1b/lib:$LD_LIBRARY_PATH
 ---> Running in 1c2d773a5c25
Removing intermediate container 1c2d773a5c25
 ---> b8a67309c446
Step 12/15 : COPY azureml-setup/spark_cache.py azureml-setup/log4j.properties /azureml-setup/
 ---> 639fe2b42e43
Step 13/15 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit "--repositories" "[]" /azureml-setup/spark_cache.py'; fi
 ---> Running in 2b8e5e18802e
Removing intermediate container 2b8e5e18802e
 ---> 76c3e4641f51
Step 14/15 : ENV AZUREML_ENVIRONMENT_IMAGE True
 ---> Running in 39fa1899d8d0
Removing intermediate container 39fa1899d8d0
 ---> 1f7288ce6b97
Step 15/15 : CMD ["bash"]
 ---> Running in 4a3e210076e4
Removing intermediate container 4a3e210076e4
 ---> a44c080ae64a
Successfully built a44c080ae64a
Successfully tagged maidaptestc9922809.azurecr.io/azureml/azureml_b2a8349416887710026a15e07f74a6a3:latest
2019/06/18 20:50:48 Successfully executed contai